In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact

def kaufkraft_prognose(
    netto_monatlich,
    aktuelles_alter,
    rentenalter,
    inflation_prozent
):
    inflationsrate = inflation_prozent / 100
    lebenserwartung = 84 + 5  # DE-Durchschnitt + Puffer

    jahre_gesamt = int(lebenserwartung - aktuelles_alter)
    alter = np.arange(aktuelles_alter, lebenserwartung + 1)

    # Entwicklung der monatlichen Kaufkraftanpassung
    kaufkraft_monatlich = [netto_monatlich * ((1 + inflationsrate) ** i) for i in range(jahre_gesamt + 1)]
    kaufkraft_jährlich = [betrag * 12 for betrag in kaufkraft_monatlich]

    # Werte bei Renteneintritt & Lebensende
    idx_rente = rentenalter - aktuelles_alter
    idx_ende = len(kaufkraft_monatlich) - 1

    netto_rente_monatlich = kaufkraft_monatlich[idx_rente]
    netto_rente_jährlich = netto_rente_monatlich * 12
    netto_ende_monatlich = kaufkraft_monatlich[idx_ende]
    netto_ende_jährlich = netto_ende_monatlich * 12

    # Plot
    plt.figure(figsize=(11, 5))
    plt.plot(alter, kaufkraft_monatlich, label="Benötigtes Netto pro Monat", linewidth=2)
    plt.axvline(rentenalter, linestyle="--", color="red", label=f"Rentenbeginn ({rentenalter})")
    plt.axvline(lebenserwartung, linestyle=":", color="gray", label=f"Lebenserwartung+5 ({lebenserwartung})")
    plt.title(
        f"Kaufkraftentwicklung bei {inflation_prozent:.1f}% Inflation\n"
        f"Rente: {netto_rente_monatlich:,.2f} €/Monat = {netto_rente_jährlich:,.2f} €/Jahr | "
        f"Am Lebensende: {netto_ende_monatlich:,.2f} €/Monat = {netto_ende_jährlich:,.2f} €/Jahr"
    )
    plt.xlabel("Alter")
    plt.ylabel("Benötigtes Netto-Einkommen (€ / Monat)")
    plt.grid(True)
    plt.legend()
    plt.show()

# Interaktives Widget
interact(
    kaufkraft_prognose,
    netto_monatlich=widgets.IntSlider(value=2500, min=500, max=10000, step=100, description="Netto/Monat (€)"),
    aktuelles_alter=widgets.IntSlider(value=28, min=18, max=70, step=1, description="Alter"),
    rentenalter=widgets.IntSlider(value=67, min=25, max=75, step=1, description="Rente mit"),
    inflation_prozent=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Inflation (%)")
);

interactive(children=(IntSlider(value=2500, description='Netto/Monat (€)', max=10000, min=500, step=100), IntS…

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def etf_rechner(
    aktuelles_alter,
    renteneintrittsalter,
    rendite_prozent,
    monatliche_einzahlung,
    erhöhung_prozent,
    entnahme_monatlich
):
    lebenserwartung = 84 + 5  # DE-Durchschnitt + 5 Jahre
    sparjahre = max(0, renteneintrittsalter - aktuelles_alter)
    entnahmejahre = max(0, lebenserwartung - renteneintrittsalter)
    
    r = rendite_prozent / 100 / 12
    re = erhöhung_prozent / 100
    n_spar = sparjahre * 12
    n_entnahme = entnahmejahre * 12
    steuer_satz = 0.18

    kapital = []
    investiert = []
    kontostand = 0

    for monat in range(1, int(n_spar) + 1):
        jahre_vergangen = monat // 12
        aktuelle_rate = monatliche_einzahlung * ((1 + re) ** jahre_vergangen)
        kontostand = kontostand * (1 + r) + aktuelle_rate
        kapital.append(kontostand)
        investiert.append(investiert[-1] + aktuelle_rate if investiert else aktuelle_rate)

    kapital_sparphase = kapital.copy()
    investiert_gesamt = investiert[-1]
    letzte_rate = monatliche_einzahlung * ((1 + re) ** (sparjahre - 1)) if sparjahre > 0 else 0

    steuerzahlungen = []
    for monat in range(1, int(n_entnahme) + 1):
        if kontostand <= 0:
            kapital.extend([0] * (n_entnahme - monat + 1))
            steuerzahlungen.extend([0] * (n_entnahme - monat + 1))
            break

        anteil_gewinn = max(0, 1 - (investiert_gesamt / kontostand))
        steuer = entnahme_monatlich * anteil_gewinn * steuer_satz
        kontostand = kontostand * (1 + r) - entnahme_monatlich - steuer
        kapital.append(kontostand)
        steuerzahlungen.append(steuer)

    vererbungssumme = max(0, kapital[-1])
    gesamt_monate = len(kapital)
    lebensjahre = np.arange(aktuelles_alter, aktuelles_alter + gesamt_monate / 12, 1 / 12)

    # Plot
    plt.figure(figsize=(11, 5))
    plt.plot(lebensjahre, kapital, label="Gesamtkapital", linewidth=2)
    if sparjahre > 0:
        plt.plot(lebensjahre[:len(kapital_sparphase)], investiert, label="Investiert", linestyle="--")
    plt.axvline(x=renteneintrittsalter, color="gray", linestyle=":", label="Rentenbeginn")
    plt.axvline(x=lebenserwartung, color="red", linestyle=":", label="Lebenserwartung + 5")
    plt.title(
        f"ETF-Rechner | Alter: {aktuelles_alter} → {int(lebenserwartung)} Jahre\n"
        f"Gesamt eingezahlt: {investiert_gesamt:,.2f} € | "
        f"Letzte Monatsrate: {letzte_rate:,.2f} € | "
        f"Steuern: {sum(steuerzahlungen):,.2f} € | "
        f"Vererbungssumme: {vererbungssumme:,.2f} €"
    )
    plt.xlabel("Alter")
    plt.ylabel("Kapital in €")
    plt.grid(True)
    plt.legend()
    plt.show()

# Interaktive Widgets
interact(
    etf_rechner,
    aktuelles_alter=widgets.IntSlider(value=28, min=18, max=70, step=1, description="Aktuelles Alter"),
    renteneintrittsalter=widgets.IntSlider(value=67, min=25, max=75, step=1, description="Rente mit"),
    rendite_prozent=widgets.FloatSlider(value=7.0, min=0, max=15, step=0.1, description="Rendite (%)"),
    monatliche_einzahlung=widgets.IntSlider(value=200, min=0, max=3000, step=50, description="Monatlich (€)"),
    erhöhung_prozent=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Dynamik (%)"),
    entnahme_monatlich=widgets.IntSlider(value=1000, min=0, max=25000, step=50, description="Entnahme/Monat (€)")
);

interactive(children=(IntSlider(value=28, description='Aktuelles Alter', max=70, min=18), IntSlider(value=67, …